In [84]:
from matplotlib import pyplot as plt
import pandas as pd
import fastf1
import fastf1.logger
import numpy as np #nie masz czasu bawic sie w optymalizje
import optuna
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from telemetry_data_preprocessing import telemetry_computations

In [85]:
##fastf1.logger.set_log_level('ERROR')
fastf1.Cache.enable_cache('cache')

In [86]:
def load_data():
    global global_df
    fastf1.Cache.enable_cache('cache')
    session = fastf1.get_session(2023, 'Bahrain', 'R')
    session.load(telemetry=True)
    laps = session.laps
    df = pd.DataFrame(laps)
    global_df = df.copy()
    df = df.dropna(subset=['LapTime', 'LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL'])
    df['LapTime_sec'] = df['LapTime'].dt.total_seconds()
    X = df[['LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL']]
    y = df['LapTime_sec']
    return X, y

x, y = load_data()

In [88]:
import fastf1
import pandas as pd

def load_data():
    global global_df

    session = fastf1.get_session(2023, 'Bahrain', 'R')
    session.load(telemetry=True)
    laps = session.laps

    global_df = laps.copy()  # zachowaj pełne dane globalnie (opcjonalnie)

    # Filtrowanie i kopiowanie
    df = laps.dropna(subset=['LapTime', 'LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL']).copy()
    df['LapTime_sec'] = df['LapTime'].dt.total_seconds()

    # Zbieranie telemetrii
    telemetry_data = []
    for _, lap in df.iterrows():
        tel = lap.get_telemetry()
        tel['LapNumber'] = lap['LapNumber']
        telemetry_data.append(tel)

    telemetry_df = pd.concat(telemetry_data, ignore_index=True)

    # Przygotowanie X i y
    X = df[['LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL']]
    y = df['LapTime_sec']
    
    return telemetry_df, X, y

telemetry, X, y = load_data()

telemetry.head(5)

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z,LapNumber
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,,0.155556,0 days 00:00:00,10076.374938,0.000000,1,16.0,False,1,interpolation,0.000194,3.748547e-08,OnTrack,-279.999631,3558.983052,-156.999885,1.0
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,,0.155556,0 days 00:00:00.110000,10130.000000,0.000000,1,16.0,False,1,car,0.000000,0.000000e+00,OnTrack,-279.999617,3558.982418,-156.999881,1.0
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,,0.155556,0 days 00:00:00.185000,9993.499884,0.000000,1,16.0,False,1,pos,-0.000268,-5.186355e-08,OnTrack,-280.000000,3559.000000,-157.000000,1.0
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,,0.155556,0 days 00:00:00.310000,9766.000000,0.000000,1,16.0,False,1,car,0.000000,0.000000e+00,OnTrack,-280.001327,3559.060983,-157.000413,1.0
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,,0.155556,0 days 00:00:00.445000,8956.001152,0.964284,1,16.0,False,1,pos,0.003325,6.439951e-07,OnTrack,-280.000000,3559.000000,-157.000000,1.0


In [89]:
telemetry.shape

(575797, 19)

In [90]:
X.head(5)

,LapNumber,SpeedI1,SpeedI2,SpeedFL
0,1.0,232.0,231.0,278.0
1,2.0,227.0,238.0,278.0
3,4.0,228.0,238.0,278.0
5,6.0,229.0,239.0,279.0
6,7.0,230.0,239.0,279.0


In [91]:
y.head(5)

0    99.019
1    97.974
3    97.976
5    97.986
6    98.021
Name: LapTime_sec, dtype: float64

In [ ]:
# def load_data():
#     global global_df
#     fastf1.Cache.enable_cache('cache')
#     session = fastf1.get_session(2023, 'Bahrain', 'R')
#     session.load(telemetry=True)
#     laps = session.laps
#     df = pd.DataFrame(laps)
#     global_df = df.copy()
#     df = df.dropna(subset=['LapTime', 'LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL'])
#     df['LapTime_sec'] = df['LapTime'].dt.total_seconds()
#     X = df[['LapNumber', 'SpeedI1', 'SpeedI2', 'SpeedFL']]
#     y = df['LapTime_sec']
#     return X, y

In [ ]:
# driver_number = "1"
# country_name = "Bahrain"
# year = 2023

# # laps = race.laps.pick_driver('VER')

# # telemetry_list = []

# # for index, lap in laps.iterlaps():
# #     telemetry = lap.get_telemetry()
# #     telemetry['LapNumber'] = lap['LapNumber']
    
# #     telemetry_list.append(telemetry)
    
# # telemetry = pd.concat(telemetry_list)

In [92]:
telemetry = telemetry.drop(["DriverAhead", "RPM", "Source", "Status", "RelativeDistance"], axis = 1)

telemetry.DRS = telemetry.DRS.apply(lambda x: 1 if x in [10, 12, 14] else 0)

In [93]:
telemetry.head(5)

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,LapNumber
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1.0
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1.0
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1.0
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1.0
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1.0


In [94]:
telemetry["mean_lap_speed"] = telemetry.groupby(by = "LapNumber")["Speed"].transform("mean")

telemetry["mean_lap_speed"] = telemetry.groupby(by = "LapNumber")["Speed"].transform("mean")

telemetry

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,LapNumber,mean_lap_speed
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1.0,177.238369
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1.0,177.238369
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1.0,177.238369
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1.0,177.238369
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1.0,177.238369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575792,2023-03-05 15:23:59.014,0 days 01:22:57.165000,14.524167,0 days 00:01:40.924000,277.000000,7,99.0,False,0,5350.113611,-390.939673,1050.884326,-158.999997,12.0,179.188233
575793,2023-03-05 15:23:59.106,0 days 01:22:57.257000,9.438625,0 days 00:01:41.016000,278.149997,7,99.0,False,0,5357.235508,-388.000000,1111.000000,-159.000000,12.0,179.188233
575794,2023-03-05 15:23:59.174,0 days 01:22:57.325000,5.679722,0 days 00:01:41.084000,279.000000,7,99.0,False,0,5362.513611,-385.457412,1163.537539,-159.000000,12.0,179.188233
575795,2023-03-05 15:23:59.186,0 days 01:22:57.337000,5.679722,0 days 00:01:41.096000,279.119997,7,99.0,False,0,5363.446582,-385.000000,1173.000000,-159.000000,12.0,179.188233


In [95]:
computations = telemetry_computations()

lon, lat = computations.compute_accelerations(telemetry=telemetry)

<class 'pandas.core.series.Series'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [96]:
telemetry["lat_acc"] = lat
telemetry["lon_acc"] = lon

In [ ]:
telemetry['TimeNumberLapTime'] = telemetry.groupby('LapNumber').cumcount() + 1
telemetry['TimeNumberLapCounts'] = telemetry.groupby('LapNumber')['LapNumber'].transform('count')

telemetry['LapProgress'] =  telemetry['TimeNumberLapTime'] / telemetry['TimeNumberLapCounts']

In [ ]:
telemetry

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,...,Z,LapNumber,mean_lap_speed,lat_acc,lon_acc,TimeNumberLapTime,TimeNumberLapCounts,LapProgress,positive_lat_acc,negative_lat_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,...,-156.999885,1.0,185.815557,0.00000,0.00000,1,762,0.001312,0.00000,0.00000
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,...,-156.999881,1.0,185.815557,0.00000,0.00000,2,762,0.002625,0.00000,0.00000
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,...,-157.000000,1.0,185.815557,-0.00000,0.00000,3,762,0.003937,-0.00000,-0.00000
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,...,-157.000413,1.0,185.815557,-0.00000,0.10502,4,762,0.005249,-0.00000,-0.00000
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,...,-157.000000,1.0,185.815557,-0.45947,0.20226,5,762,0.006562,0.00000,-0.45947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29921,2023-03-05 16:37:34.828,0 days 02:36:32.979000,NaN,0 days 00:01:35.733000,271.000000,7,0.0,True,0,5322.452222,...,-158.999991,57.0,200.748140,-0.06975,-0.73090,738,742,0.994609,0.00000,-0.06975
29922,2023-03-05 16:37:35.005,0 days 02:36:33.156000,NaN,0 days 00:01:35.910000,263.256253,7,0.0,True,0,5335.352916,...,-159.000000,57.0,200.748140,-0.07614,-1.23882,739,742,0.995957,0.00000,-0.07614
29923,2023-03-05 16:37:35.148,0 days 02:36:33.299000,NaN,0 days 00:01:36.053000,257.000000,7,0.0,True,0,5345.296667,...,-159.000000,57.0,200.748140,0.11348,-1.45304,740,742,0.997305,0.11348,0.00000
29924,2023-03-05 16:37:35.185,0 days 02:36:33.336000,NaN,0 days 00:01:36.090000,254.019450,7,0.0,True,0,5347.787384,...,-159.000000,57.0,200.748140,0.11467,-2.28099,741,742,0.998652,0.11467,0.00000


In [ ]:
def divide_column_by_sign(df : pd.DataFrame, column : str) -> pd.DataFrame:
    
    df[f"positive_{column}"] = df[column].apply(lambda x: 0 if x < 0 else x)
    df[f"negative_{column}"] = df[column].apply(lambda x: 0 if x > 0 else x)
    
    return df


telemetry = divide_column_by_sign(telemetry, "lat_acc")

telemetry

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,...,Z,LapNumber,mean_lap_speed,lat_acc,lon_acc,TimeNumberLapTime,TimeNumberLapCounts,LapProgress,positive_lat_acc,negative_lat_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,...,-156.999885,1.0,185.815557,0.00000,0.00000,1,762,0.001312,0.00000,0.00000
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,...,-156.999881,1.0,185.815557,0.00000,0.00000,2,762,0.002625,0.00000,0.00000
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,...,-157.000000,1.0,185.815557,-0.00000,0.00000,3,762,0.003937,-0.00000,-0.00000
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,...,-157.000413,1.0,185.815557,-0.00000,0.10502,4,762,0.005249,-0.00000,-0.00000
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,...,-157.000000,1.0,185.815557,-0.45947,0.20226,5,762,0.006562,0.00000,-0.45947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29921,2023-03-05 16:37:34.828,0 days 02:36:32.979000,NaN,0 days 00:01:35.733000,271.000000,7,0.0,True,0,5322.452222,...,-158.999991,57.0,200.748140,-0.06975,-0.73090,738,742,0.994609,0.00000,-0.06975
29922,2023-03-05 16:37:35.005,0 days 02:36:33.156000,NaN,0 days 00:01:35.910000,263.256253,7,0.0,True,0,5335.352916,...,-159.000000,57.0,200.748140,-0.07614,-1.23882,739,742,0.995957,0.00000,-0.07614
29923,2023-03-05 16:37:35.148,0 days 02:36:33.299000,NaN,0 days 00:01:36.053000,257.000000,7,0.0,True,0,5345.296667,...,-159.000000,57.0,200.748140,0.11348,-1.45304,740,742,0.997305,0.11348,0.00000
29924,2023-03-05 16:37:35.185,0 days 02:36:33.336000,NaN,0 days 00:01:36.090000,254.019450,7,0.0,True,0,5347.787384,...,-159.000000,57.0,200.748140,0.11467,-2.28099,741,742,0.998652,0.11467,0.00000


Obliczanie submodelu telemetrii w konteście całego okrążenia

In [ ]:
lap_df =  telemetry.copy()

lap_df['abs_lat_acc'] = lap_df['lat_acc'].abs()
lap_df['sum_lat_acc'] = lap_df.groupby('LapNumber')['abs_lat_acc'].transform('sum')

lap_df['abs_lon_acc'] = lap_df['lon_acc'].abs()
lap_df['sum_lon_acc'] = lap_df.groupby('LapNumber')['abs_lon_acc'].transform('sum')


In [ ]:
lap_df

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,...,lon_acc,TimeNumberLapTime,TimeNumberLapCounts,LapProgress,positive_lat_acc,negative_lat_acc,abs_lat_acc,sum_lat_acc,abs_lon_acc,sum_lon_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,...,0.00000,1,762,0.001312,0.00000,0.00000,0.00000,840.62035,0.00000,560.87748
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,...,0.00000,2,762,0.002625,0.00000,0.00000,0.00000,840.62035,0.00000,560.87748
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,...,0.00000,3,762,0.003937,-0.00000,-0.00000,0.00000,840.62035,0.00000,560.87748
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,...,0.10502,4,762,0.005249,-0.00000,-0.00000,0.00000,840.62035,0.10502,560.87748
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,...,0.20226,5,762,0.006562,0.00000,-0.45947,0.45947,840.62035,0.20226,560.87748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29921,2023-03-05 16:37:34.828,0 days 02:36:32.979000,NaN,0 days 00:01:35.733000,271.000000,7,0.0,True,0,5322.452222,...,-0.73090,738,742,0.994609,0.00000,-0.06975,0.06975,859.40488,0.73090,553.94680
29922,2023-03-05 16:37:35.005,0 days 02:36:33.156000,NaN,0 days 00:01:35.910000,263.256253,7,0.0,True,0,5335.352916,...,-1.23882,739,742,0.995957,0.00000,-0.07614,0.07614,859.40488,1.23882,553.94680
29923,2023-03-05 16:37:35.148,0 days 02:36:33.299000,NaN,0 days 00:01:36.053000,257.000000,7,0.0,True,0,5345.296667,...,-1.45304,740,742,0.997305,0.11348,0.00000,0.11348,859.40488,1.45304,553.94680
29924,2023-03-05 16:37:35.185,0 days 02:36:33.336000,NaN,0 days 00:01:36.090000,254.019450,7,0.0,True,0,5347.787384,...,-2.28099,741,742,0.998652,0.11467,0.00000,0.11467,859.40488,2.28099,553.94680


In [ ]:
lap_df.drop(columns=['lon_acc', 'lat_acc', 'positive_lat_acc', 'negative_lat_acc', 'abs_lat_acc', 'abs_lon_acc', 'LapProgress'], inplace=True)

In [ ]:
lap_df

,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,LapNumber,mean_lap_speed,TimeNumberLapTime,TimeNumberLapCounts,sum_lat_acc,sum_lon_acc
0,2023-03-05 15:03:38.501,0 days 01:02:36.652000,0.155556,0 days 00:00:00,0.000000,1,16.0,False,0,0.000194,-279.999631,3558.983052,-156.999885,1.0,185.815557,1,762,840.62035,560.87748
1,2023-03-05 15:03:38.611,0 days 01:02:36.762000,0.155556,0 days 00:00:00.110000,0.000000,1,16.0,False,0,0.000000,-279.999617,3558.982418,-156.999881,1.0,185.815557,2,762,840.62035,560.87748
2,2023-03-05 15:03:38.686,0 days 01:02:36.837000,0.155556,0 days 00:00:00.185000,0.000000,1,16.0,False,0,-0.000268,-280.000000,3559.000000,-157.000000,1.0,185.815557,3,762,840.62035,560.87748
3,2023-03-05 15:03:38.811,0 days 01:02:36.962000,0.155556,0 days 00:00:00.310000,0.000000,1,16.0,False,0,0.000000,-280.001327,3559.060983,-157.000413,1.0,185.815557,4,762,840.62035,560.87748
4,2023-03-05 15:03:38.946,0 days 01:02:37.097000,0.155556,0 days 00:00:00.445000,0.964284,1,16.0,False,0,0.003325,-280.000000,3559.000000,-157.000000,1.0,185.815557,5,762,840.62035,560.87748
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29921,2023-03-05 16:37:34.828,0 days 02:36:32.979000,NaN,0 days 00:01:35.733000,271.000000,7,0.0,True,0,5322.452222,-407.146267,742.650673,-158.999991,57.0,200.748140,738,742,859.40488,553.94680
29922,2023-03-05 16:37:35.005,0 days 02:36:33.156000,NaN,0 days 00:01:35.910000,263.256253,7,0.0,True,0,5335.352916,-400.000000,881.000000,-159.000000,57.0,200.748140,739,742,859.40488,553.94680
29923,2023-03-05 16:37:35.148,0 days 02:36:33.299000,NaN,0 days 00:01:36.053000,257.000000,7,0.0,True,0,5345.296667,-394.385089,984.164803,-159.000000,57.0,200.748140,740,742,859.40488,553.94680
29924,2023-03-05 16:37:35.185,0 days 02:36:33.336000,NaN,0 days 00:01:36.090000,254.019450,7,0.0,True,0,5347.787384,-393.000000,1010.000000,-159.000000,57.0,200.748140,741,742,859.40488,553.94680


In [ ]:
def get_single_lap_data(df: pd.DataFrame) -> pd.DataFrame:
    final_df = pd.DataFrame(columns=df.columns)
    
    laps = int(df['LapNumber'].max())
    for i in range(1, laps + 1):  # Adjust range to include maximum lap number
        lap_df = df[df['LapNumber'] == i]
        if not lap_df.empty:
            final_row = lap_df.iloc[[-1],:]
            final_df = pd.concat([final_df, final_row], axis=0)
    
    return final_df

acc_df = get_single_lap_data(lap_df)

acc_df

C:\Users\Patryk\AppData\Local\Temp\ipykernel_15632\2664467517.py:9: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, final_row], axis=0)


,Date,SessionTime,DistanceToDriverAhead,Time,Speed,nGear,Throttle,Brake,DRS,Distance,X,Y,Z,LapNumber,mean_lap_speed,TimeNumberLapTime,TimeNumberLapCounts,sum_lat_acc,sum_lon_acc
761,2023-03-05 15:05:17.751,0 days 01:04:15.902000,0.000278,0 days 00:01:39.250000,280.142857,7,100.0,False,0,5141.964903,-383.313167,1216.856742,-159.000000,1.0,185.815557,762,762,840.62035,560.87748
1499,2023-03-05 15:06:55.725,0 days 01:05:53.876000,NaN,0 days 00:01:37.974000,280.941666,7,100.0,False,0,5363.501061,-383.071911,1222.720301,-158.999999,2.0,196.531157,738,738,856.02736,514.79313
2258,2023-03-05 15:10:11.707,0 days 01:09:09.858000,NaN,0 days 00:01:37.976000,280.435716,7,100.0,False,0,5360.387868,-382.866368,1223.632581,-158.999999,4.0,197.711395,759,759,850.09057,511.85026
2984,2023-03-05 15:13:27.728,0 days 01:12:25.879000,NaN,0 days 00:01:37.986000,281.299169,7,100.0,False,0,5369.135413,-382.806640,1217.956630,-158.971744,6.0,197.696016,726,726,798.41078,505.85609
3714,2023-03-05 15:15:05.749,0 days 01:14:03.900000,NaN,0 days 00:01:38.021000,280.885714,7,100.0,False,0,5369.141600,-382.807112,1225.243520,-159.000006,7.0,197.638955,730,730,819.26118,502.92284
4435,2023-03-05 15:18:22.181,0 days 01:17:20.332000,NaN,0 days 00:01:38.278000,280.469998,7,100.0,False,0,5357.558452,-383.107053,1221.224106,-159.000000,9.0,198.031115,721,721,801.54246,491.95707
5164,2023-03-05 15:20:00.550,0 days 01:18:58.701000,NaN,0 days 00:01:38.369000,281.742857,7,100.0,False,0,5360.503372,-382.651356,1232.568831,-159.000022,10.0,198.022196,729,729,786.45545,501.00287
5904,2023-03-05 15:23:17.624,0 days 01:22:15.775000,NaN,0 days 00:01:38.591000,280.806250,7,100.0,False,0,5351.839787,-381.981249,1244.013410,-159.000000,12.0,196.354889,740,740,814.78850,511.22157
6649,2023-03-05 15:24:56.106,0 days 01:23:54.257000,NaN,0 days 00:01:38.482000,281.603123,7,100.0,False,0,5348.555270,-382.543554,1229.037919,-159.000000,13.0,197.694001,745,745,824.22662,524.98674
7536,2023-03-05 15:28:35.779,0 days 01:27:33.930000,716.727500,0 days 00:01:58.378000,281.329177,7,100.0,False,0,5379.813391,-380.853447,1261.003994,-159.000000,15.0,163.356430,887,887,867.44933,542.52337


In [ ]:
X_acc = acc_df.loc[:, ['LapNumber', 'sum_lat_acc', 'sum_lon_acc']]

In [ ]:
X_c = pd.merge(X, X_acc, on='LapNumber',)

In [ ]:
X_c

,LapNumber,SpeedI1,SpeedI2,SpeedFL,sum_lat_acc,sum_lon_acc
0,1.0,232.0,231.0,278.0,840.62035,560.87748
1,2.0,227.0,238.0,278.0,856.02736,514.79313
2,4.0,228.0,238.0,278.0,850.09057,511.85026
3,6.0,229.0,239.0,279.0,798.41078,505.85609
4,7.0,230.0,239.0,279.0,819.26118,502.92284
5,9.0,229.0,242.0,279.0,801.54246,491.95707
6,10.0,230.0,242.0,279.0,786.45545,501.00287
7,12.0,231.0,241.0,278.0,814.78850,511.22157
8,13.0,230.0,241.0,278.0,824.22662,524.98674
9,15.0,230.0,237.0,279.0,867.44933,542.52337


In [ ]:
y

0      99.019
1      97.974
3      97.976
5      97.986
6      98.021
8      98.278
9      98.369
11     98.591
12     98.482
14    118.378
15     97.801
16     97.648
19     97.788
20     97.582
23     97.387
24     97.372
26     97.379
27     97.651
28     97.571
30     97.431
32     97.686
33     97.839
36    118.490
37     96.297
38     96.446
39     96.942
40    120.495
41    104.824
42     96.890
43     96.236
44     97.015
45     96.701
48     96.962
50     97.427
51     96.919
52     96.890
53     96.678
54     96.322
55     97.205
56     96.373
Name: LapTime_sec, dtype: float64

In [ ]:
assert X_c.shape[0] == y.shape[0]

Linear Regression

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
print(f"Linear Regression MSE: {mse_linear:.3f}")


scaler = StandardScaler()
X_scaled_c = scaler.fit_transform(X_c)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_scaled_c, y, test_size=0.2)

# Linear Regression
linear_model = LinearRegression()
linear_model.fit(X_train_c, y_train_c)
y_pred_linear_c = linear_model.predict(X_test_c)
mse_linear = mean_squared_error(y_test_c, y_pred_linear_c)
print(f"Linear Regression MSE with additional data: {mse_linear:.3f}")

Linear Regression MSE: 20.763
Linear Regression MSE with additional data: 2.158


Decision Tree


In [ ]:
tree_model = DecisionTreeRegressor(random_state=12)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
print(f"Decision Tree MSE: {mse_tree:.3f}")

tree_model = DecisionTreeRegressor(random_state=12)
tree_model.fit(X_train_c, y_train_c)
y_pred_tree_c = tree_model.predict(X_test_c)
mse_tree = mean_squared_error(y_test_c, y_pred_tree_c)
print(f"Decision Tree MSE with additional data: {mse_tree:.3f}")

Obliczanie submodelu telemetrii w konteście podziału na sektory

In [ ]:
def compute_binned(df : pd.DataFrame, column : str = 'lat', bins : int = 10, aggfunc : str ='mean') -> pd.DataFrame:
    
    df['bin'] = pd.cut(df['LapProgress'], bins=bins, labels=False, include_lowest=True)
    
    result = (
        df.groupby(['LapNumber', 'bin'])[column]
        .agg(aggfunc)
        .reset_index()
        .rename(columns={column: f'{aggfunc}_of_{column}'})
    )
    return result

In [ ]:
binned_result = compute_binned(telemetry, column='lat_acc', bins=10, aggfunc='sum')

binned_result

,LapNumber,bin,sum_of_lat_acc
0,1.0,0,-1.40
1,1.0,1,-27.03
2,1.0,2,-79.70
3,1.0,3,-1.87
4,1.0,4,-30.47
...,...,...,...
565,57.0,5,60.15
566,57.0,6,110.83
567,57.0,7,-165.74
568,57.0,8,-3.58


Podejścia:

suma z sektorów porównana do:

- średniej kroczącej z innych okrążeń

- top okrążenia w stincie

- top okrążenia kwalifikacyjnego
